In [202]:
%pip install pandas statsmodels seaborn matplotlib scikit-learn


[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [203]:
# インポート
import pandas as pd
import statsmodels.api as sm
from statsmodels.tools import add_constant
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set()

# JSONLファイルのパス
jsonl_file_path = 'listings_2024_sales.json'  # ご自身のファイルパスに置き換えてください

# データをDataFrameに変換
df = pd.read_json(jsonl_file_path, lines=True)

# データの確認
print("データの最初の数行:")
print(df.head())

データの最初の数行:
  manual_id listing_name                          room_type_id owner_name  \
0       1-1     月村ビル 302  f39a7eb0-f29a-46f6-b9b4-9b053e4aefb7         MT   
1       1-2     月村ビル 303  777c7031-af8d-4ccc-bece-5aa8da1f4487         MT   
2       1-3     月村ビル 401  4b3eabab-36f7-4cdf-9d30-03feb80d5ea9         MT   
3       1-4     月村ビル 403  13125f4f-874e-400b-9615-fd425af877a7         MT   
4       7-1    アズール渋谷幡ヶ谷  3fc60dce-589e-4e51-8b45-4e5bc19a7278         MT   

  stay_operation_type prefecture_name city_name floor_plan    sqm  \
0                  住宿             東京都       杉並区        2DK  35.50   
1                  住宿             東京都       杉並区        2DK  42.64   
2                  住宿             東京都       杉並区        2DK  42.64   
3                  住宿             東京都       杉並区        2DK  42.64   
4                 旅館業             東京都       渋谷区        3DK  66.93   

   number_of_capacity has_elevator has_auto_lock first_line first_station  \
0                 6.0            無

In [204]:
# データの基本情報
print("\nデータの基本情報:")
print(df.info())

print("\n統計概要:")
print(df.describe())

print("\n欠損値の数:")
print(df.isnull().sum())



データの基本情報:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2234 entries, 0 to 2233
Data columns (total 31 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   manual_id            2234 non-null   object 
 1   listing_name         2234 non-null   object 
 2   room_type_id         2234 non-null   object 
 3   owner_name           2234 non-null   object 
 4   stay_operation_type  2234 non-null   object 
 5   prefecture_name      2234 non-null   object 
 6   city_name            2234 non-null   object 
 7   floor_plan           2233 non-null   object 
 8   sqm                  2231 non-null   float64
 9   number_of_capacity   2228 non-null   float64
 10  has_elevator         343 non-null    object 
 11  has_auto_lock        338 non-null    object 
 12  first_line           2227 non-null   object 
 13  first_station        2227 non-null   object 
 14  first_walk_min       2227 non-null   object 
 15  location_floor       2143 n

/Users/ishibashi_y/workspace/rm-analyze-listingsales/lib/python3.12/site-packages/pandas/core/nanops.py:1016: RuntimeWarning: invalid value encountered in subtract
  sqr = _ensure_numeric((avg - values) ** 2)
/Users/ishibashi_y/workspace/rm-analyze-listingsales/lib/python3.12/site-packages/pandas/core/nanops.py:1016: RuntimeWarning: invalid value encountered in subtract
  sqr = _ensure_numeric((avg - values) ** 2)


In [205]:
# 欠損値の処理

df['number_of_s_beds'].fillna(0, inplace=True)
df['number_of_sd_beds'].fillna(0, inplace=True)
df['number_of_d_beds'].fillna(0, inplace=True)
df['number_of_q_beds'].fillna(0, inplace=True)
df['number_of_k_beds'].fillna(0, inplace=True)
df['number_of_futons'].fillna(0, inplace=True)
df['number_of_sofa_beds'].fillna(0, inplace=True)

df['first_walk_min'] = pd.to_numeric(df['first_walk_min'], errors='coerce')

df['has_elevator'] = df['has_elevator'].map({'有': 1, '無': 0})
df['has_elevator'].fillna(0, inplace=True)

df['has_auto_lock'] = df['has_auto_lock'].map({'有': 1, '無': 0})
df['has_auto_lock'].fillna(0, inplace=True)

df = pd.get_dummies(df, columns=[
    'stay_operation_type', 'prefecture_name', 'city_name', 'floor_plan', 'tag', 'first_station'
], drop_first=True)

df = df.dropna()

print(df.head())

# 再度欠損値を確認
print("\n欠損値の補完後:")
print(df.isnull().sum())



  manual_id listing_name                          room_type_id owner_name  \
0       1-1     月村ビル 302  f39a7eb0-f29a-46f6-b9b4-9b053e4aefb7         MT   
1       1-2     月村ビル 303  777c7031-af8d-4ccc-bece-5aa8da1f4487         MT   
2       1-3     月村ビル 401  4b3eabab-36f7-4cdf-9d30-03feb80d5ea9         MT   
3       1-4     月村ビル 403  13125f4f-874e-400b-9615-fd425af877a7         MT   
8      10-1  シャトレ西大井 202  9859e29a-edc0-4bdc-870d-b005708fd022         MT   

     sqm  number_of_capacity  has_elevator  has_auto_lock first_line  \
0  35.50                 6.0           0.0            0.0     京王井の頭線   
1  42.64                 8.0           0.0            0.0     京王井の頭線   
2  42.64                 8.0           0.0            0.0     京王井の頭線   
3  42.64                 8.0           0.0            0.0     京王井の頭線   
8  39.75                 5.0           0.0            0.0  JR湘南新宿ライン   

   first_walk_min  location_floor  built_year  number_of_s_beds  \
0             2.0             3.0    

/var/folders/pd/2jj0t6nx237d_97r8zs2969c0000gn/T/ipykernel_2442/1093562982.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['number_of_s_beds'].fillna(0, inplace=True)
/var/folders/pd/2jj0t6nx237d_97r8zs2969c0000gn/T/ipykernel_2442/1093562982.py:4: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always b

In [206]:
# 説明変数の定義
features = [
    'sqm',
    'number_of_capacity',
    'has_elevator',
    'has_auto_lock',
    'location_floor',
    'built_year',
    'number_of_sd_beds', 'number_of_d_beds', 'number_of_q_beds', 'number_of_k_beds', 'number_of_futons', 'number_of_sofa_beds',
]

# 東京だけに絞る
df = df[df['prefecture_name_東京都'] == True]
X = df[features]


In [207]:
# 相関係数を確認
cor = df[features].corr()

cor


,sqm,number_of_capacity,has_elevator,has_auto_lock,location_floor,built_year,number_of_sd_beds,number_of_d_beds,number_of_q_beds,number_of_k_beds,number_of_futons,number_of_sofa_beds
sqm,1.000000,0.873954,-0.029681,0.081011,0.128239,0.152443,0.035949,0.132327,-0.015865,-0.002568,-0.088531,0.084957
number_of_capacity,0.873954,1.000000,0.048490,0.065033,0.099963,0.025941,0.178373,0.209823,-0.022501,0.016746,-0.008528,0.125007
has_elevator,-0.029681,0.048490,1.000000,0.671319,0.174258,-0.412392,0.353983,0.571615,-0.011749,0.062445,-0.004149,0.266951
has_auto_lock,0.081011,0.065033,0.671319,1.000000,0.212061,-0.091761,0.068502,0.606324,0.078536,0.078536,0.132862,0.385789
location_floor,0.128239,0.099963,0.174258,0.212061,1.000000,-0.007328,0.006792,0.126638,0.005662,-0.007296,-0.068182,0.047000
built_year,0.152443,0.025941,-0.412392,-0.091761,-0.007328,1.000000,-0.363173,-0.302097,0.007479,-0.059310,-0.138279,-0.126015
number_of_sd_beds,0.035949,0.178373,0.353983,0.068502,0.006792,-0.363173,1.000000,0.014785,-0.005707,-0.005707,0.033339,0.013076
number_of_d_beds,0.132327,0.209823,0.571615,0.606324,0.126638,-0.302097,0.014785,1.000000,-0.009193,-0.009193,0.103073,0.365095
number_of_q_beds,-0.015865,-0.022501,-0.011749,0.078536,0.005662,0.007479,-0.005707,-0.009193,1.000000,-0.000734,-0.003631,-0.004833
number_of_k_beds,-0.002568,0.016746,0.062445,0.078536,-0.007296,-0.059310,-0.005707,-0.009193,-0.000734,1.000000,-0.003631,-0.004833


In [208]:
# VIFの計算

import pandas as pd
from statsmodels.stats.outliers_influence import variance_inflation_factor

# VIFの計算
vif_data = pd.DataFrame()
vif_data["feature"] = X.columns
vif_data["VIF"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]

vif_data


,feature,VIF
0,sqm,33.344643
1,number_of_capacity,32.435990
2,has_elevator,3.183177
3,has_auto_lock,2.875690
4,location_floor,4.204905
5,built_year,9.421325
6,number_of_sd_beds,1.460515
7,number_of_d_beds,2.254725
8,number_of_q_beds,1.020338
9,number_of_k_beds,1.021383


In [209]:
high_vif = vif_data[vif_data['VIF'] > 10]['feature']

X = X.drop(columns=high_vif)


print("VIFが10を超える変数:")
high_vif

VIFが10を超える変数:


0                   sqm
1    number_of_capacity
Name: feature, dtype: object

In [210]:
# 特徴量とターゲットの選択
target = 'total_adr'

X = X.astype(float)

y = df[target]

X = add_constant(X)

# 線形回帰モデルの構築
model = sm.OLS(y, X).fit()

# モデルの要約を表示
print("\nモデルの要約:")
model.summary()



モデルの要約:


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              total_adr   R-squared:                       0.068
Model:                            OLS   Adj. R-squared:                  0.061
Method:                 Least Squares   F-statistic:                     9.926
Date:                Tue, 05 Nov 2024   Prob (F-statistic):           3.32e-16
Time:                        16:01:12   Log-Likelihood:                -13458.
No. Observations:                1364   AIC:                         2.694e+04
Df Residuals:                    1353   BIC:                         2.700e+04
Df Model:                          10                                         
Covariance Type:            nonrobust                                         
=======================================================================================
                          coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------
const               -1.104e+05    2.3e+04     -4.811      0.000   -1.55e+05   -6.54e+04
has_elevator        -2858.4728    590.549     -4.840      0.000   -4016.964   -1699.982
has_auto_lock         684.2194    681.893      1.003      0.316    -653.463    2021.902
location_floor        192.9454     62.514      3.086      0.002      70.310     315.581
built_year             59.4330     11.372      5.226      0.000      37.124      81.742
number_of_sd_beds    1156.8476    458.804      2.521      0.012     256.803    2056.892
number_of_d_beds     1558.7843    478.306      3.259      0.001     620.483    2497.085
number_of_q_beds    -3352.8543   4733.286     -0.708      0.479   -1.26e+04    5932.521
number_of_k_beds     1674.1855   4738.103      0.353      0.724   -7620.641     1.1e+04
number_of_futons    -1892.2732    715.474     -2.645      0.008   -3295.832    -488.715
number_of_sofa_beds   336.2400    777.188      0.433      0.665   -1188.384    1860.864
==============================================================================
Omnibus:                      978.023   Durbin-Watson:                   0.897
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            16769.591
Skew:                           3.180   Prob(JB):                         0.00
Kurtosis:                      18.956   Cond. No.                     3.65e+05
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 3.65e+05. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [211]:
# Ridge回帰のR^2を計算

from sklearn.linear_model import Ridge
from sklearn.model_selection import cross_val_score

ridge = Ridge(alpha=1.0)
scores = cross_val_score(ridge, X_reduced, y, cv=5, scoring='r2')
print("Ridge回帰の平均R²:", scores.mean())

Ridge回帰の平均R²: -0.11630645918997282
